In [173]:
from selenium import webdriver
import time
import pandas as pd
import numpy as np

#relevant libraries to webscraping
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException

from selenium.webdriver.common.by import By
from selenium.common.exceptions import ElementNotInteractableException


In [14]:
import sys
sys.path.append('C:\\Users\\mario\\python')

In [177]:

url = 'https://www.linkedin.com/jobs/search/?currentJobId=3671506295&f_E=4&f_TPR=r86400&geoId=90000084&keywords=data%20scientist&location=San%20Francisco%20Bay%20Area&origin=JOB_SEARCH_PAGE_JOB_FILTER&refresh=true&sortBy=R'
path_driver = "C:\\Users\\mario\\python\\chromedriver.exe"
s = Service(path_driver)

driver = webdriver.Chrome(service=s)
driver.get(url)


In [178]:
job_count_text = driver.find_element(By.CLASS_NAME, "results-context-header__job-count").text
# Remove commas and plus symbol
clean_text = job_count_text.replace(',', '').replace('+', '')

# Convert to integer
no_of_jobs = int(clean_text)
print(no_of_jobs)


330


In [179]:
#loading all jobs

i = 2
while i <= int(no_of_jobs/25)+1:
    
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    i = i + 1
    
    try:
        driver.find_element(By.XPATH, '/html/body/div[1]/div/main/section[2]/button').click()
        time.sleep(5)
    except:
        pass
        time.sleep(5)

In [180]:
job_title = []
hiring_company = []
job_location = []
job_link = []
recruiter = []
description = []

i =1

for count in range(1,no_of_jobs):
    
    print(f'job number: {i}')
    
    try:
        # Extract the text from the located element
        path = f'/html/body/div[1]/div/main/section[2]/ul/li[{i}]/div/a'
        driver.find_element(By.XPATH, path).click()

    except NoSuchElementException:
        # If the element is not present, set the salary_text to np.nan
        path = f'/html/body/div[1]/div/main/section[2]/ul/li[{i}]/a'
        driver.find_element(By.XPATH, path).click() 
    time.sleep(10)
    
    
    try:
        
        driver.find_element(By.CSS_SELECTOR, 'button[data-tracking-control-name="public_jobs_show-more-html-btn"]').click()
        time.sleep(5)

        
        title = driver.find_element(By.CSS_SELECTOR, '.top-card-layout__title').text

        try:
            company = driver.find_element(By.CSS_SELECTOR, '.topcard__flavor a').text
        except NoSuchElementException:
            company = driver.find_element(By.CSS_SELECTOR, '.topcard__flavor').text

        location = driver.find_element(By.CSS_SELECTOR, ".topcard__flavor-row .topcard__flavor--bullet").text
        link = driver.find_element(By.XPATH, "//a[@class='topcard__link']").get_attribute('href')
        has_message_recruiter = driver.find_elements(By.CSS_SELECTOR, ".message-the-recruiter.message-the-recruiter--jserp")

        if len(has_message_recruiter) > 0:
            has_message_recruiter = True
        else:
            has_message_recruiter = False

        job_description = driver.find_element(By.CSS_SELECTOR, '.show-more-less-html__markup').text

        job_title.append(title)
        hiring_company.append(company)
        job_location.append(location)
        job_link.append(link)
        recruiter.append(has_message_recruiter)
        description.append(job_description)

        i += 1
        
    except ElementNotInteractableException:
        
        job_title.append(np.nan)
        hiring_company.append(np.nan)
        job_location.append(np.nan)
        job_link.append(np.nan)
        recruiter.append(np.nan)
        description.append(np.nan)
        
        print('no click show more error')
        
        i += 1


job number: 1
job number: 2
job number: 3
job number: 4
job number: 5
job number: 6
job number: 7
job number: 8
job number: 9
job number: 10
job number: 11
job number: 12
job number: 13
job number: 14
job number: 15
job number: 16
job number: 17
job number: 18
job number: 19
job number: 20
job number: 21
job number: 22
job number: 23
job number: 24
job number: 25
job number: 26
job number: 27
job number: 28
job number: 29
job number: 30
job number: 31
job number: 32
job number: 33
job number: 34
job number: 35
job number: 36
job number: 37
job number: 38
job number: 39
job number: 40
job number: 41
job number: 42
job number: 43
job number: 44
job number: 45
job number: 46
job number: 47
job number: 48
job number: 49
job number: 50
job number: 51
no click show more error
job number: 52
job number: 53
job number: 54
job number: 55
job number: 56
job number: 57
job number: 58
job number: 59
job number: 60
job number: 61
job number: 62
job number: 63
job number: 64
job number: 65
job numbe

In [165]:
driver.find_element(By.CSS_SELECTOR, 'button[data-tracking-control-name="public_jobs_show-more-html-btn"]').click()

In [181]:
df_job_posts = pd.DataFrame({    
'job_title' : job_title,
'hiring_company' : hiring_company,
'job_location' : job_location,
'job_link' : job_link,
'recruiter' : recruiter,
'description' : description
})


len(job_title)

329

In [182]:
df_job_posts.to_csv('linkedin_jobs_second_try.csv')

### other alternative for the metadata

In [68]:

job_list = driver.find_element(By.CLASS_NAME,'two-pane-serp-page__results-list')
jobs = job_list.find_elements(By.TAG_NAME,'li')
i = 4

title = jobs[i].find_element(By.CSS_SELECTOR, '.base-search-card__title').text
company = jobs[i].find_element(By.CSS_SELECTOR, '.base-search-card__info .base-search-card__subtitle a').text
location = jobs[i].find_element(By.CSS_SELECTOR, '.job-search-card__location').text

#datetime
try:
    posted_date =  jobs[i].find_element(By.CSS_SELECTOR, '.job-search-card__listdate').get_attribute('datetime')
except NoSuchElementException:
    try:
        # If the first attempt fails, try the second CSS selector
        posted_date =  jobs[i].find_element(By.CSS_SELECTOR, 'job-search-card__listdate--new').get_attribute('datetime')
    except NoSuchElementException:
        # If both attempts fail, set the salary_text to np.nan
        salary_text = np.nan

#salary
try:
    # Extract the text from the located element
    salary_text = jobs[i].find_element(By.CSS_SELECTOR, '.job-search-card__salary-info').text

except NoSuchElementException:
    # If the element is not present, set the salary_text to np.nan
    salary_text = np.nan

    
    
entity_urn = jobs[i].find_element(By.CSS_SELECTOR, 'li > div').get_attribute('data-entity-urn')
job_id = entity_urn.split(':')[-1] if entity_urn else None


nan


In [175]:

job_title.append(np.nan)

In [176]:
job_title

['Data Scientist- TikTok Ads, Ads Targeting, Auction and Delivery',
 'Computational Scientist – Human Gentics',
 'Founding Scientists',
 'Sr. Principal Scientist - Data Analytics and Pharmacometric Modeling',
 'Scientist, Bioinformatics Engineer',
 'Scientist',
 'Computational Biologist',
 'Clinical Lab Scientist III',
 'Senior Clinical Data Scientist',
 'Talent Scientist',
 'Director, Analytics Engineer - Biopharma',
 'Lead / Principal Clinical Data Scientist',
 'Clinical Lab Scientist IV',
 'Scientist/Project Scientist - Oceanography/Marine Scientist',
 'Scientist/Project Scientist - Oceanography/Marine Scientist',
 'Scientist/Project Scientist - Oceanography/Marine Scientist',
 'Senior Clinical Programmer',
 'Filemaker developer',
 'Research Associate II, Assay Strategy and Development',
 'Scientist II or Senior Scientist I, Computational Oncology',
 'Clinical Microbiologist Scientist',
 'Senior Principal Scientist, Machine Learning, Molecule Prediction and Design',
 'Sr. Scientist 

In [184]:
import os
print(os.getcwd())


C:\Users\mario


In [108]:

i = 1
path = f'/html/body/div[1]/div/main/section[2]/ul/li[{i}]/div/a'

driver.find_element(By.XPATH, path).click()
time.sleep(5)
driver.find_element(By.CSS_SELECTOR, 'button[data-tracking-control-name="public_jobs_show-more-html-btn"]').click()
time.sleep(5)


title = driver.find_element(By.CSS_SELECTOR, '.top-card-layout__title').text
company = driver.find_element(By.CSS_SELECTOR, '.topcard__flavor a').text
location = driver.find_element(By.CSS_SELECTOR, ".topcard__flavor-row .topcard__flavor--bullet").text
link = driver.find_element(By.XPATH, "//a[@class='topcard__link']").get_attribute('href')
has_message_recruiter = driver.find_elements(By.CSS_SELECTOR, ".message-the-recruiter.message-the-recruiter--jserp")

if len(has_message_recruiter) > 0:
    has_message_recruiter = True
else:
    has_message_recruiter = False

job_description = driver.find_element(By.CSS_SELECTOR, '.show-more-less-html__markup').text



The goal of the ML team at Scale is to develop machine learning solutions advancing the company mission. Our current focus areas are computer vision (2D/3D detection, 2D/3D segmentation, object tracking), meta-learning (e.g. semi-supervised learning, active learning), and Natural Language Processing. You’ll be working on a combination of deeply technical ML applications in production and cutting edge research problems. Working at Scale will give you opportunities to work with our wide customer base which includes leading research teams and exposure to a wide range of problems within machine learning.

We are building a large hybrid human-machine system in service of ML pipelines for dozens of industry-leading customers. Our machine learning models form the basis for Scale’s expansion and future product strategy. We currently complete billions of tasks a month, and will continue to grow to support more complex use cases and more advanced ML powered products.

Example Projects:

Research

'Machine Learning Research Engineer, New Grad'

'Scale AI'

'San Francisco, CA'

'https://www.linkedin.com/jobs/view/machine-learning-research-engineer-new-grad-at-scale-ai-3698001044?refId=Y%2FpBDNkXs9JJiEL5LsuV8g%3D%3D&trackingId=qBSHikQxWVK4iP9QQ9fbiQ%3D%3D&trk=public_jobs_topcard-title'

False

In [ ]:
from selenium import webdriver
import time
import pandas as pd
import numpy as np

#relevant libraries to webscraping
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException

from selenium.webdriver.common.by import By

import sys
sys.path.append('C:\\Users\\mario\\python')

job_count_text = driver.find_element(By.CLASS_NAME, "results-context-header__job-count").text
# Remove commas and plus symbol
clean_text = job_count_text.replace(',', '').replace('+', '')

# Convert to integer
no_of_jobs = int(clean_text)
print(no_of_jobs)

#loading all jobs

i = 2
while i <= int(no_of_jobs/25)+1:
    
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    i = i + 1
    
    try:
        driver.find_element(By.XPATH, '/html/body/div[1]/div/main/section[2]/button').click()
        time.sleep(5)
    except:
        pass
        time.sleep(5)
        
job_title = []
hiring_company = []
job_location = []
job_link = []
recruiter = []
description = []

for i in range(1,no_of_jobs):
    
    try:
        # Extract the text from the located element
        path = f'/html/body/div[1]/div/main/section[2]/ul/li[{i}]/div/a'
        driver.find_element(By.XPATH, path).click()

    except NoSuchElementException:
        # If the element is not present, set the salary_text to np.nan
        path = f'/html/body/div[1]/div/main/section[2]/ul/li[{i}]/a'
        driver.find_element(By.XPATH, path).click() 
    time.sleep(10)
    
    driver.find_element(By.CSS_SELECTOR, 'button[data-tracking-control-name="public_jobs_show-more-html-btn"]').click()
    time.sleep(5)

    title = driver.find_element(By.CSS_SELECTOR, '.top-card-layout__title').text

    try:
        company = driver.find_element(By.CSS_SELECTOR, '.topcard__flavor a').text
    except NoSuchElementException:
        company = driver.find_element(By.CSS_SELECTOR, '.topcard__flavor').text
    
    location = driver.find_element(By.CSS_SELECTOR, ".topcard__flavor-row .topcard__flavor--bullet").text
    link = driver.find_element(By.XPATH, "//a[@class='topcard__link']").get_attribute('href')
    has_message_recruiter = driver.find_elements(By.CSS_SELECTOR, ".message-the-recruiter.message-the-recruiter--jserp")

    if len(has_message_recruiter) > 0:
        has_message_recruiter = True
    else:
        has_message_recruiter = False

    job_description = driver.find_element(By.CSS_SELECTOR, '.show-more-less-html__markup').text
    
    job_title.append(title)
    hiring_company.append(company)
    job_location.append(location)
    job_link.append(link)
    recruiter.append(has_message_recruiter)
    description.append(job_description)

df_job_posts = pd.DataFrame({    
'job_title' : job_title,
'hiring_company' : hiring_company,
'job_location' : job_location,
'job_link' : job_link,
'recruiter' : recruiter,
'description' : description
})


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"tag name","selector":"show-more-less-html__markup relative overflow-hidden"}
  (Session info: chrome=117.0.5938.92); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7996F7892+54818]
	(No symbol) [0x00007FF799666AC2]
	(No symbol) [0x00007FF79951DA3B]
	(No symbol) [0x00007FF79955E4FC]
	(No symbol) [0x00007FF79955E67C]
	(No symbol) [0x00007FF799599627]
	(No symbol) [0x00007FF79957EAEF]
	(No symbol) [0x00007FF7995975A2]
	(No symbol) [0x00007FF79957E883]
	(No symbol) [0x00007FF799553691]
	(No symbol) [0x00007FF7995548D4]
	GetHandleVerifier [0x00007FF799A5B992+3610402]
	GetHandleVerifier [0x00007FF799AB1860+3962352]
	GetHandleVerifier [0x00007FF799AA9D4F+3930847]
	GetHandleVerifier [0x00007FF799793646+693206]
	(No symbol) [0x00007FF799671628]
	(No symbol) [0x00007FF79966